# 1) Loading fiqa-pl corpus

In [77]:
from datasets import load_dataset

ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 57638
    })
})

In [78]:
corpus = ds['corpus']
corpus

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

# 2) Preparing elasticsearch

Let's define a link for a local elasticsearch, beacuase i disabled ssl in elastic config we do not need to link to certificate

In [79]:
link = 'http://localhost:9200/'

In [80]:
from elasticsearch import Elasticsearch

es = Elasticsearch(link)
es

<Elasticsearch(['http://localhost:9200'])>

Now let's define the index 
- First we need analyzer one with synonym filter and one without. Both have to have lowercase and morfologik_stem filter
- Secondly filter that defines the synonyms for Polish month `kwiecień`
- Thirdly mappings one for each anylzer

In [81]:
index_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                "polish_with_synonyms": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "synonym_filter",
                        "morfologik_stem"
                    ]
                },
                "polish_without_synonyms": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem"
                    ]
                }
            },
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms": ["kwiecień, kwi, IV"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content_with_synonyms": {"type": "text", "analyzer": "polish_with_synonyms"},
            "content_without_synonyms": {"type": "text", "analyzer": "polish_without_synonyms"}
        }
    }
}


Now we need to prepare texts/documents for elastic indexing. We loop through corpus and index every text. Thanks to `helpers.bulk()` function we can efficiently index multiple documents in a single API call.

In [82]:
from elasticsearch import helpers

index_name =  "biore_sie_do_roboty"

def generate_docs(ds):
    for doc in ds:
        yield {
            "_index": index_name,
            "_id": doc["_id"],
            "_source": {
                "content_with_synonyms": doc["text"], 
                "content_without_synonyms": doc["text"]
            }
        }


Let's also delete and add the index so that we always create a new on

In [84]:
es.options(ignore_status=[400, 404]).indices.delete(index=index_name)
es.indices.create(index=index_name, body=index_config)

try:
    success, errors = helpers.bulk(es, generate_docs(corpus))
    
    print(f"Successfully indexed {success} documents")
    if errors:
        print(f"Errors during indexing: {errors}")
        
except Exception as e:
    print(f"Error during bulk indexing: {e}")

Successfully indexed 57638 documents


Now let's define the queries for `kwiecień` and see how many times it occures in corpus with and without synonyms

In [89]:
query_with_synonyms = {
    "query": {
        "match": {
            "content_with_synonyms": "kwiecień"
        }
    }
}

query_without_synonyms = {
    "query": {
        "match": {
            "content_without_synonyms": "kwiecień"
        }
    }
}

response = es.search(index=index_name, body=query_with_synonyms)
print("The number of occurrences of the word 'kwiecień' including its synonyms:",response['hits']['total']['value'])
response = es.search(index=index_name, body=query_without_synonyms)
print("The number of occurrences of the word 'kwiecień' without synonyms:",response['hits']['total']['value'])


The number of occurrences of the word 'kwiecień' including its synonyms: 306
The number of occurrences of the word 'kwiecień' without synonyms: 257


# 3) Using fiqa-pl-qrels dataset

### a) Preparing the qa dataset

In [164]:
ds2 = load_dataset("clarin-knext/fiqa-pl-qrels")
ds2

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 14166
    })
    validation: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1238
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1706
    })
})

In [165]:
ds3 = load_dataset("clarin-knext/fiqa-pl", "queries")
queries = ds3['queries']
queries

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 6648
})

In [166]:
corpus_dict = {int(doc['_id']): doc['text'] for doc in corpus}
queries_dict = {int(query['_id']): query['text'] for query in queries}

In [170]:
corpus_dict[3]

'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'

In [172]:
queries_dict[0]

'Co jest uważane za wydatek służbowy w podróży służbowej?'

### b) prepering the lemmatization indexes

In [173]:
index_config

{'settings': {'analysis': {'analyzer': {'polish_with_synonyms': {'tokenizer': 'standard',
     'filter': ['lowercase', 'synonym_filter', 'morfologik_stem']},
    'polish_without_synonyms': {'tokenizer': 'standard',
     'filter': ['lowercase', 'morfologik_stem']}},
   'filter': {'synonym_filter': {'type': 'synonym',
     'synonyms': ['kwiecień, kwi, IV']}}}},
 'mappings': {'properties': {'content_with_synonyms': {'type': 'text',
    'analyzer': 'polish_with_synonyms'},
   'content_without_synonyms': {'type': 'text',
    'analyzer': 'polish_without_synonyms'}}}}

In [174]:
index_config_lamentizer = {
    "settings": {
        "analysis": {
            "analyzer": {
                "polish_with_synonyms_lam": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "synonym_filter",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "polish_without_synonyms_lam": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                }
            },
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms": ["kwiecień, kwi, IV"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content_with_synonyms_lam": {"type": "text", "analyzer": "polish_with_synonyms_lam"},
            "content_without_synonyms_lam": {"type": "text", "analyzer": "polish_without_synonyms_lam"}
        }
    }
}

In [175]:
index_name_lam = index_name + '_lam'

In [152]:
def generate_docs_lam(ds):
    for doc in ds:
        yield {
            "_index": index_name_lam,
            "_id": doc["_id"],
            "_source": {
                "content_with_synonyms_lam": doc['question'] + " " + doc["answer"],
                "content_without_synonyms_lam": doc['question'] + " " + doc["answer"],
            }
        }


In [155]:
es.options(ignore_status=[400, 404]).indices.delete(index=index_name_lam)
es.indices.create(index=index_name_lam, body=index_config_lamentizer)

try:
    success, errors = helpers.bulk(es, generate_docs_lam(combined))
    
    print(f"Successfully indexed {success} documents")
    if errors:
        print(f"Errors during indexing: {errors}")
        
except Exception as e:
    print(f"Error during bulk indexing: {e}")

Successfully indexed 1706 documents


In [157]:
query_with_synonyms = {
    "query": {
        "match": {
            "content_with_synonyms_lam": "kwiecień"
        }
    }
}

query_without_synonyms = {
    "query": {
        "match": {
            "content_without_synonyms_lam": "kwiecień"
        }
    }
}

response = es.search(index=index_name_lam, body=query_with_synonyms)
print("(Lamentaizer) The number of occurrences of the word 'kwiecień' including its synonyms:",response['hits']['total']['value'])
response = es.search(index=index_name_lam, body=query_without_synonyms)
print("(Lamentaizer) The number of occurrences of the word 'kwiecień' without synonyms:",response['hits']['total']['value'])


(Lamentaizer) The number of occurrences of the word 'kwiecień' including its synonyms: 19
(Lamentaizer) The number of occurrences of the word 'kwiecień' without synonyms: 16


In [158]:
def generate_docs_qa(ds):
    for doc in ds:
        yield {
            "_index": index_name,
            "_id": doc["_id"],
            "_source": {
                "content_with_synonyms": doc['question'] + " " + doc["answer"],
                "content_without_synonyms": doc['question'] + " " + doc["answer"],
            }
        }


In [159]:
es.options(ignore_status=[400, 404]).indices.delete(index=index_name_lam)
es.indices.create(index=index_name_lam, body=index_config_lamentizer)

try:
    success, errors = helpers.bulk(es, generate_docs_qa(combined))
    
    print(f"Successfully indexed {success} documents")
    if errors:
        print(f"Errors during indexing: {errors}")
        
except Exception as e:
    print(f"Error during bulk indexing: {e}")

Successfully indexed 1706 documents


In [160]:
query_with_synonyms = {
    "query": {
        "match": {
            "content_with_synonyms": "kwiecień"
        }
    }
}

query_without_synonyms = {
    "query": {
        "match": {
            "content_without_synonyms": "kwiecień"
        }
    }
}

response = es.search(index=index_name, body=query_with_synonyms)
print("(No Lamantizer) The number of occurrences of the word 'kwiecień' including its synonyms:",response['hits']['total']['value'])
response = es.search(index=index_name, body=query_without_synonyms)
print("(No Lamantizer) The number of occurrences of the word 'kwiecień' without synonyms:",response['hits']['total']['value'])


(No Lamantizer) The number of occurrences of the word 'kwiecień' including its synonyms: 324
(No Lamantizer) The number of occurrences of the word 'kwiecień' without synonyms: 272


In [161]:
response

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 272, 'relation': 'eq'}, 'max_score': 10.497359, 'hits': [{'_index': 'biore_sie_do_roboty', '_id': '533589', '_score': 10.497359, '_source': {'content_with_synonyms': 'Załóżmy, że w przeszłości płaciłeś odsetki od poprzednich opłat. Twój miesięczny wyciąg jest wystawiany 12 kwietnia i (ponieważ właśnie otrzymałeś zwrot podatku dochodowego), spłacasz go w całości 30 kwietnia. Po 12 kwietnia w ogóle nie obciążasz karty. 30, saldo Twojej karty kredytowej pokazuje zero, ponieważ właśnie je spłaciłeś. Ale wyciąg z 12 kwietnia naliczał odsetki tylko do 12 kwietnia. Tak więc 12 maja następny miesięczny rachunek będzie dotyczył odsetek za saldo niezerowe od 13 do 30 kwietnia. Zakładając, że nadal nie naliczasz żadnych nowych opłat na swojej karcie i spłacaj w terminie rachunek z 12 maja, w końcu 12 czerwca będziesz miał rachunek zerowy. Co się stanie

In [162]:
from datasets import load_dataset

ds2 = load_dataset("clarin-knext/fiqa-pl-qrels")
ds_corpus = ds
ds_queries = load_dataset("clarin-knext/fiqa-pl", "queries")

questions = {int(el["_id"]): el["text"] for el in ds_queries["queries"] if int(el["_id"]) in ds2["test"]["query-id"]}
answers = {int(el["_id"]): el["text"] for el in ds_corpus["corpus"] if int(el["_id"]) in ds2["test"]["corpus-id"]}

In [163]:
questions

{4641: 'Gdzie powinienem zaparkować mój fundusz na deszczowy dzień / awaryjny?',
 5503: 'Względy podatkowe związane ze sprzedażą rodzinie nieruchomości poniżej szacunkowej wartości?',
 7803: 'Czy Delta może być wykorzystana do obliczenia premii opcyjnej przy określonym celu?',
 7017: 'Podstawowa strategia handlu algorytmicznego',
 10152: 'Co oznacza dla firmy wysoka marża operacyjna, ale niewielkie, ale dodatnie ROE?',
 3451: 'Czy powinieneś zatrzymać swoje zapasy, jeśli spóźnisz się na sprzedaż?',
 4804: 'Czym usługi finansowe skierowane do kobiet różnią się od usług konwencjonalnych?',
 7911: 'Jaka jest różnica między „handlowcem” a „maklerem giełdowym”?',
 10809: 'Definicje dźwigni finansowej i współczynnika dźwigni',
 6715: 'Co to znaczy, jeśli „IPO – zwykle są sprzedawane z „rabatem ubezpieczeniowym” (wbudowana prowizja)”',
 2388: 'Czy doradcy finansowi uzyskują lepsze oferty na kredyty hipoteczne?',
 753: 'Podatki należne dla hobbystów Group Buy',
 4465: 'Jak przekazać darowiznę 